In [26]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [27]:
book=pd.read_csv('books.csv')

In [28]:
book.head(10)

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9.780000e+12,2005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9.780000e+12,2261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9.780010e+12,6163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9.780010e+12,6178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9.780010e+12,6280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0
5,9.780010e+12,6280935,The Problem of Pain,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0
6,9.780010e+12,6353282,An Autobiography,NaN,Agatha Christie,"Authors, English",http://books.google.com/books/content?id=c49GQ...,Donation.,1977.0,4.27,560.0,3975.0
7,9.780010e+12,6380832,Empires of the Monsoon,A History of the Indian Ocean and Its Invaders,Richard Hall,"Africa, East",http://books.google.com/books/content?id=MuPEQ...,Until Vasco da Gama discovered the sea-route t...,1998.0,4.41,608.0,65.0
8,9.780010e+12,000647022X,The Gap Into Madness,Chaos and Order,Stephen R. Donaldson,"Hyland, Morn (Fictitious character)",http://books.google.com/books/content?id=4oXav...,A new-cover reissue of the fourth book in the ...,1994.0,4.15,743.0,103.0
9,9.780010e+12,6472613,Master of the Game,NaN,Sidney Sheldon,Adventure stories,http://books.google.com/books/content?id=TkTYp...,Kate Blackwell is an enigma and one of the mos...,1982.0,4.11,489.0,43540.0


In [29]:
book.describe()

,isbn13,published_year,average_rating,num_pages,ratings_count
count,6.809000e+03,6803.000000,6766.000000,6766.000000,6.766000e+03
mean,9.780677e+12,1998.630163,3.933314,348.184747,2.107210e+04
std,6.068316e+08,10.485015,0.331367,242.394503,1.376306e+05
min,9.780000e+12,1853.000000,0.000000,0.000000,0.000000e+00
25%,9.780330e+12,1996.000000,3.770000,208.000000,1.590000e+02
50%,9.780550e+12,2002.000000,3.960000,304.000000,1.018000e+03
75%,9.780810e+12,2005.000000,4.130000,420.000000,5.993250e+03
max,9.789040e+12,2019.000000,5.000000,3342.000000,5.629932e+06


In [30]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6809 entries, 0 to 6808
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   isbn13          6809 non-null   float64
 1   isbn10          6809 non-null   object 
 2   title           6809 non-null   object 
 3   subtitle        2381 non-null   object 
 4   authors         6737 non-null   object 
 5   categories      6710 non-null   object 
 6   thumbnail       6480 non-null   object 
 7   description     6547 non-null   object 
 8   published_year  6803 non-null   float64
 9   average_rating  6766 non-null   float64
 10  num_pages       6766 non-null   float64
 11  ratings_count   6766 non-null   float64
dtypes: float64(5), object(7)
memory usage: 638.5+ KB


In [31]:
book.isnull().sum()

isbn13               0
isbn10               0
title                0
subtitle          4428
authors             72
categories          99
thumbnail          329
description        262
published_year       6
average_rating      43
num_pages           43
ratings_count       43
dtype: int64

In [32]:
book['isbn13']

0       9.780000e+12
1       9.780000e+12
2       9.780010e+12
3       9.780010e+12
4       9.780010e+12
            ...     
6804    9.788190e+12
6805    9.788190e+12
6806    9.788450e+12
6807    9.789030e+12
6808    9.789040e+12
Name: isbn13, Length: 6809, dtype: float64

In [33]:
book['title']

0                          Gilead
1                    Spider's Web
2                    The One Tree
3                  Rage of angels
4                  The Four Loves
                  ...            
6804                    I Am that
6805         Secrets Of The Heart
6806               Fahrenheit 451
6807     The Berlin Phenomenology
6808    'I'm Telling You Stories'
Name: title, Length: 6809, dtype: object

In [34]:
book=book[['isbn13', 'title', 'description', 'categories']]

In [35]:
book

,isbn13,title,description,categories
0,9.780000e+12,Gilead,A NOVEL THAT READERS and critics have been eag...,Fiction
1,9.780000e+12,Spider's Web,A new 'Christie for Christmas' -- a full-lengt...,Detective and mystery stories
2,9.780010e+12,The One Tree,Volume Two of Stephen Donaldson's acclaimed se...,American fiction
3,9.780010e+12,Rage of angels,"A memorable, mesmerizing heroine Jennifer -- b...",Fiction
4,9.780010e+12,The Four Loves,Lewis' work on the nature of love divides love...,Christian life
...,...,...,...,...
6804,9.788190e+12,I Am that,This collection of the timeless teachings of o...,Philosophy
6805,9.788190e+12,Secrets Of The Heart,NaN,Mysticism
6806,9.788450e+12,Fahrenheit 451,NaN,Book burning
6807,9.789030e+12,The Berlin Phenomenology,Since the three volume edition ofHegel's Philo...,History


In [36]:
book['features'] = book['description']+book['categories']

In [37]:
book

,isbn13,title,description,categories,features
0,9.780000e+12,Gilead,A NOVEL THAT READERS and critics have been eag...,Fiction,A NOVEL THAT READERS and critics have been eag...
1,9.780000e+12,Spider's Web,A new 'Christie for Christmas' -- a full-lengt...,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...
2,9.780010e+12,The One Tree,Volume Two of Stephen Donaldson's acclaimed se...,American fiction,Volume Two of Stephen Donaldson's acclaimed se...
3,9.780010e+12,Rage of angels,"A memorable, mesmerizing heroine Jennifer -- b...",Fiction,"A memorable, mesmerizing heroine Jennifer -- b..."
4,9.780010e+12,The Four Loves,Lewis' work on the nature of love divides love...,Christian life,Lewis' work on the nature of love divides love...
...,...,...,...,...,...
6804,9.788190e+12,I Am that,This collection of the timeless teachings of o...,Philosophy,This collection of the timeless teachings of o...
6805,9.788190e+12,Secrets Of The Heart,NaN,Mysticism,NaN
6806,9.788450e+12,Fahrenheit 451,NaN,Book burning,NaN
6807,9.789030e+12,The Berlin Phenomenology,Since the three volume edition ofHegel's Philo...,History,Since the three volume edition ofHegel's Philo...


In [38]:
book  = book.drop(columns=['description','categories'])

In [39]:
# Replace missing values with an empty string
book['features'] = book['features'].fillna('')

In [40]:
book

,isbn13,title,features
0,9.780000e+12,Gilead,A NOVEL THAT READERS and critics have been eag...
1,9.780000e+12,Spider's Web,A new 'Christie for Christmas' -- a full-lengt...
2,9.780010e+12,The One Tree,Volume Two of Stephen Donaldson's acclaimed se...
3,9.780010e+12,Rage of angels,"A memorable, mesmerizing heroine Jennifer -- b..."
4,9.780010e+12,The Four Loves,Lewis' work on the nature of love divides love...
...,...,...,...
6804,9.788190e+12,I Am that,This collection of the timeless teachings of o...
6805,9.788190e+12,Secrets Of The Heart,
6806,9.788450e+12,Fahrenheit 451,
6807,9.789030e+12,The Berlin Phenomenology,Since the three volume edition ofHegel's Philo...


In [41]:

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(book['features'])


In [42]:
# Apply Non-Negative Matrix Factorization (NMF)
n_components = 50  # Set the desired number of latent factors
nmf = NMF(n_components=n_components)
latent_matrix = nmf.fit_transform(tfidf_matrix)


In [43]:

# Calculate the cosine similarity between latent factors
latent_similarity = cosine_similarity(latent_matrix)



In [44]:
print(latent_similarity)

[[1.         0.15900245 0.07796316 ... 0.         0.01270135 0.0240646 ]
 [0.15900245 1.         0.04230901 ... 0.         0.10032717 0.03381106]
 [0.07796316 0.04230901 1.         ... 0.         0.18799171 0.00523005]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01270135 0.10032717 0.18799171 ... 0.         1.         0.00775118]
 [0.0240646  0.03381106 0.00523005 ... 0.         0.00775118 1.        ]]


In [45]:
# Function to recommend movies based on similarity
def recommend_book(book_title, top_n=5):
    # Get the index of the movie
    idx = book[book['title'] == book_title].index[0]
    
    # Get the pairwise similarity scores
    similarity_scores = list(enumerate(latent_similarity[idx]))
    
    # Sort movies based on similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get top similar movies
    top_book = [book.iloc[i[0]]['title'] for i in similarity_scores[1:top_n+1]]
    
    return top_book

In [49]:
# Example usage for NMF recommendation
recommendations_nmf = recommend_book("The One Tree", top_n=5)
print(recommendations_nmf)

['Letters Home', "King Solomon's Mines", 'The Known World', 'Black Sunday', 'Echo Platoon']


In [47]:
# Save movies and latent_similarity to pickle files
pickle.dump(book, open('books.pkl', 'wb'))
pickle.dump(latent_similarity, open('book_latent_similarity.pkl', 'wb'))

# Load movies and latent_similarity from pickle files
book = pickle.load(open('books.pkl', 'rb'))
latent_similarity = pickle.load(open('book_latent_similarity.pkl', 'rb'))